In [ ]:
from IPython.display import Image

from segar.mdps import MDP
from segar.sim import Simulator
from segar.tasks.classic_control import CartPoleInitialization, CartPoleTask, CartPoleObservation, from_cartpole_basis
from segar.tools.sample_trajectories import rollout, save_gif